### Imports

In [1]:
import numpy as np
from math import log
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint
from itertools import zip_longest
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

### Whether all the data in a attribute is of same type

In [2]:
def pure_data(data):
    label_column = data[:, -1]
    u_val = np.unique(label_column)
    if len(u_val) == 1:
        return True
    else:
        return False

In [3]:
def classifyy(data):
    label_column = data[:, -1]
    u_val, counts_u_val = np.unique(label_column, return_counts=True)
    index = counts_u_val.argmax()
    classification = u_val[index]
#     print(u_val,"-" ,counts_u_val)
    return classification

### To find the value to split data in 2 halves

In [4]:
def find_best_val_to_split(data):
    best_attr_split = {}
    _, n_columns = data.shape
    # print( _, n_columns)
    for cols in range(n_columns - 1):  
        values = data[:, cols]
        u_valss = np.unique(values)
        type_of_feature = FEATURE_TYPES[cols]
        if type_of_feature == "continuous":
            best_attr_split[cols] = []
            for index in range(len(u_valss)):
                if index != 0:
                    current_value = u_valss[index]
                    previous_value = u_valss[index - 1]
                    split_val = (current_value + previous_value) / 2
                    best_attr_split[cols].append(split_val)

        elif len(u_valss) > 1:
            best_attr_split[cols] = u_valss
    return best_attr_split

### Splitting data into 2 parts to form tree structure

In [5]:
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]
    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        df_left = data[split_column_values <= split_value]
        df_right = data[split_column_values >  split_value]
    # feature is categorical   
    else:
        df_left = data[split_column_values == split_value]
        df_right = data[split_column_values != split_value]
    
    return df_left, df_right   

### To find lowest entropy of columns

In [6]:
# def misclassification(data,label):
#     label_column = data[:, -1]
#     _, counts = np.unique(label_column, return_counts=True)

#     q_value = counts / counts.sum()
#     entropy = -999
#     for x in q_value:
#         if entropy < x:
#             entropy = x
#     return entropy

# def gini_index(data,label):
#     label_column = data[:, -1]
#     _, counts = np.unique(label_column, return_counts=True)

#     q_value = counts / counts.sum()
#     entropy = 0
#     for x in q_value:
#         entropy = x * x
#     return entropy

def calculate_entropy(data,label):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)
    q_value = counts / counts.sum()
    entropy = sum(q_value * -np.log2(q_value))
    return entropy

    # Gini index
    # entropy=2*q_value*(1-q_value)
    # return entropy[0]

    # misclassification rate
    # entropy=min(q_value[0],1-q_value[0])
    # return entropy

In [7]:
def total_entropy(df_left, df_right1):
    df_right = df_right1[:]
    n = len(df_left) + len(df_right)
    d_left = len(df_left) / n
    d_right = len(df_right) / n

    # tot_entr =  (d_left * calculate_entropy(df_left) 
    #                   + d_right * calculate_entropy(df_right))
    tot_entr =  (d_left * calculate_entropy(df_left,label) 
                      + d_right * calculate_entropy(df_right,label))
    
    return tot_entr

In [8]:
def determine_best_split(data, best_attr_split):
    tot_entr = 9999
    for cols in best_attr_split:
        for value in best_attr_split[cols]:
            df_left, df_right = split_data(data, split_column=cols, split_value=value)
            current_tot_entr = total_entropy(df_left, df_right)

            if current_tot_entr <= tot_entr:
                tot_entr = current_tot_entr
                col_break = cols
                value_break = value
    
    return col_break, value_break                         

In [13]:
def determine_type_of_feature(df):
    feature_types = []
    threshold_for_categorical = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]
            if (isinstance(example_value, str)) or (len(unique_values) <= threshold_for_categorical):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [14]:
def Make_dec_tree(df, counter=0, min_samples=2, max_depth=5):
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    if (pure_data(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classifyy(data)
        return classification

    else:    
        counter += 1
        best_attr_split = find_best_val_to_split(data)
        split_column, split_value = determine_best_split(data, best_attr_split)
        data_below, data_above = split_data(data, split_column, split_value)
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
        else:
            question = "{} = {}".format(feature_name, split_value)
        sub_tree = {question: []}
        yes_answer = Make_dec_tree(data_below, counter, min_samples, max_depth)
        no_answer = Make_dec_tree(data_above, counter, min_samples, max_depth)
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
    
def classification_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, compare, value = question.split(" ")

    if compare == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    if not isinstance(answer, dict):
        return answer
    
    else:
        rest_tree = answer
        return classification_example(example, rest_tree)


def all_calculations(df, tree):
    correct,TP,FP,FN,TN=0,0,0,0,0
    # global correct,TP,FP,FN
    df["classification"] = df.apply(classification_example, args=(tree,), axis=1)
    # print(df["classification"])
    for value, left in zip_longest(df["classification"],df['left']):
        if value == left:
            if left == 0:
                TN += 1
            else:
                TP += 1
    else:
        if left == 0:
            FP += 1
        else:
            FN += 1
            
    x = (TP+FN)
    y = (TP+FP)
    if x:
        rc=TP/x
    if y:
        pc=TP/y
    f1=(2*rc*pc)/(rc+pc)
    df["classification_correct"] = df["classification"] == df["left"]  
    accuracy = df["classification_correct"].mean()
    return accuracy,TP,FP,FN,TN,rc,pc,f1

def predicts(df, tree):
    df["classification"] = df.apply(classification_example, args=(tree,), axis=1)
    # print(df["classification"])
#     df["classification_correct"] = df["classification"] == df["left"]  
    return df["classification"] 

df = pd.read_csv("../input_data/train1.csv")
# df = df.rename(columns={"left": "label"})
# random.seed(0)
label = "left"

train_df, test_df = train_test_split(df, test_size=0.2)
tree = Make_dec_tree(train_df, max_depth=25)
accuracy,TP,FP,FN,TN,rc,pc,f1= all_calculations(test_df, tree)
# TP = calculate_accuracy(test_df, tree)
# FP = calculate_accuracy(test_df, tree)
# FN = calculate_accuracy(test_df, tree)
pp = predicts(test_df, tree)
# print(pp)
# question = list(tree.keys())[0]
# print(question)
pprint(tree, width=100)
# print(tree)
print("\nThe accuracy is: ", accuracy)
# print(TP)
# print(FP)
# print(FN)
# print(TN)
print("\nRecall: ", rc)
print("\nPrecision: ", pc)
print("\nF1-Score: ", f1)




/home/keshu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/keshu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'satisfaction_level <= 0.46499999999999997': [{'satisfaction_level <= 0.11499999999999999': [1,
                                                                                              {'number_project = 2': [{'time_spend_company = 3': [{'satisfaction_level <= 0.355': [0,
                                                                                                                                                                                   {'average_montly_hours <= 125.0': [0,
                                                                                                                                                                                                                      {'average_montly_hours <= 163.0': [{'last_evaluation <= 0.6299999999999999': [{'salary = high': [{'sales = IT': [0,
                                                                                                                                                                              

/home/keshu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Accuracy using Scikit

In [16]:
#Comparing result with in-built(scikit-learn) decision tree function to check correctness of algorithm used
df = pd.read_csv("../input_data/train1.csv")
from sklearn import tree
model = tree.DecisionTreeClassifier()
le_salary = LabelEncoder()
le_sales=LabelEncoder()
df['sales_n'] = le_salary.fit_transform(df['sales'])
df['salary_n'] = le_sales.fit_transform(df['salary'])
df=df.drop(['sales','salary'],axis='columns')

#dividing the data into training and testing data(for validation)
msk = np.random.rand(len(df)) < 0.8
train2 = df[msk]
test2 = df[~msk]
trainy=train2['left']
trainx=train2.drop(['left'],axis='columns')

#training the model
model.fit(trainx,trainy)
testy=test2['left']
testx=test2.drop(['left'],axis='columns')

#predicting over the test data
pred=model.predict(testx)
print( accuracy_score(testy,pred))
print (confusion_matrix(testy,pred))
print (classification_report(testy,pred))

0.9746059544658494
[[1696   37]
 [  21  530]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1733
           1       0.93      0.96      0.95       551

   micro avg       0.97      0.97      0.97      2284
   macro avg       0.96      0.97      0.97      2284
weighted avg       0.97      0.97      0.97      2284

